# NanoPyx "Codeless" Jupyter Notebook

### This notebook showcases loading an example datast with NanoPyx, performing super-resolution image generation using SRRF (Super-Resolution Radiality Fluctuations) and then measuring quality control metrics such as FRC (Fourier Ring Correlation) and Decorrelation analysis.
  
### To use this notebook you don't need to interact with any code, just run cells in order and a graphical user interface will pop-up showcasing the parameters for each step
  
**SRRF**: Culley S, Tosheva KL, Matos Pereira P, Henriques R. SRRF: Universal live-cell super-resolution microscopy. Int J Biochem Cell Biol. 2018 Aug;101:74-79. doi: 10.1016/j.biocel.2018.05.014. Epub 2018 May 28. PMID: 29852248; PMCID: PMC6025290.  
  
**FRC**: Nieuwenhuizen RP, Lidke KA, Bates M, Puig DL, Grünwald D, Stallinga S, Rieger B. Measuring image resolution in optical nanoscopy. Nat Methods. 2013 Jun;10(6):557-62. doi: 10.1038/nmeth.2448. Epub 2013 Apr 28. PMID: 23624665; PMCID: PMC4149789.  
  
**DecorrAnalysis**: Descloux A, Grußmayer KS, Radenovic A. Parameter-free image resolution estimation based on decorrelation analysis. Nat Methods. 2019 Sep;16(9):918-924. doi: 10.1038/s41592-019-0515-7. Epub 2019 Aug 26. PMID: 31451766.  

## Notebook setup cell
Run the following cell to install nanopyx

In [ ]:
!pip install git+https://github.com/HenriquesLab/NanoPyx.git
!pip install stackview

Now, import all necessary python libraries and start the gui_data_data setup.

In [ ]:
import os
import stackview

import ipywidgets as widgets
import skimage
from IPython.display import display

from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files
from nanopyx.data.download import ExampleDataManager

cwd = os.getcwd()
image_folder = "datasets"
image_files = []
EDM = ExampleDataManager()
example_datasets = EDM.list_datasets()

_path = os.path.join("..", image_folder)
if os.path.exists(_path):
    image_files += find_files(_path, ".tif")
if os.path.exists(image_folder):
    image_files += find_files(image_folder, ".tif")
image_files += ["Example dataset: "+dataset for dataset in example_datasets]

## Next lets create the Data Loader GUI.

Once you run the cell, make sure to select the SMLMS2013_HDTubulingAlexa647 dataset

In [ ]:
# Create a GUI
gui_data = EasyGui("Data Loader")

def on_button_load_data_clicked(b):
    global dataset_original
    # disable button
    gui_data["load_data"].disabled = True
    gui_data["load_data"].description = "Loading..."

    if gui_data["data_source"].value.startswith("Example dataset: "):
        dataset_name = gui_data["data_source"].value.replace(
            "Example dataset: ", "")
        dataset_original = EDM.get_ZipTiffIterator(dataset_name, as_ndarray=True)
    else:
        dataset_original = skimage.io.imread(gui_data["data_source"].value)
    
    # enable button
    gui_data["load_data"].disabled = False
    gui_data["load_data"].description = "Load data"
    gui_data.save_settings()

gui_data.add_label("Select data to use:")
gui_data.add_dropdown("data_source", options=image_files,
                 value="Example dataset: "+example_datasets[4], remember_value=True)
gui_data.add_button("load_data", description="Load data")
gui_data["load_data"].on_click(on_button_load_data_clicked)
gui_data.show()

## Let's take a look at the dataset

In [ ]:
stackview.slice(dataset_original, continuous_update=True, zoom_factor=2, slider_text="Frame")

## Now let's use SRRF to generate a super-resolution image

In [ ]:
gui_srrf = EasyGui("srrf")
from nanopyx.methods.srrf import SRRF

def run_srrf(b):
    gui_srrf.save_settings()
    ring_radius = gui_srrf["ring_radius"].value
    magnification = gui_srrf["magnification"].value
    frames_per_timepoint = gui_srrf["frames_per_timepoint"].value
    srrf_order = gui_srrf["srrf_order"].value
    # disable button while running
    gui_srrf["run"].disabled = True
    gui_srrf["run"].description = "Running..."
    srrf = SRRF(magnification, ring_radius)
    global dataset_srrf
    dataset_srrf = srrf.calculate(dataset_original, frames_per_timepoint, srrf_order)
    # enable button again
    gui_srrf["run"].disabled = False
    gui_srrf["run"].description = "Run"

gui_srrf.add_float_slider("ring_radius", description="Ring Radius:", min=0.1, max=3.0, value=0.5, remember_value=True)
gui_srrf.add_int_slider("magnification", description="Magnification:", min=1, max=10, value=5)
gui_srrf.add_int_slider("srrf_order", description="SRRF order:", min=-1, max=4, value=3)
gui_srrf.add_label("-=-= Time-Lapse =-=-")
gui_srrf.add_int_slider("frames_per_timepoint", description="Frames per time-point (0 - auto)", min=1, max=dataset_original.shape[0], value=dataset_original.shape[0]//2)
gui_srrf.add_button("run", description="Run")
gui_srrf['run'].on_click(run_srrf)
gui_srrf.show()

In [ ]:
stackview.curtain(dataset_srrf[0], dataset_srrf[1], continuous_update=True, zoom_factor=1, zoom_spline_order=0)

## Let's use NanoPyx to generate an error map of the SRRF image

In [ ]:
gui_error = EasyGui("Error")

import numpy as np
from matplotlib import pyplot as plt
from nanopyx.core.transform.new_error_map import ErrorMap

def run_error(b):
    gui_error.save_settings()
    gui_error["run"].disabled = True
    gui_error["run"].description = "Calculating..."
    global error_map
    error_map = ErrorMap()
    error_map.optimise(np.mean(dataset_original, axis=0), np.mean(dataset_srrf[0], axis=0))
    gui_error["run"].disabled = False
    gui_error["run"].description = "Calculate"
    print("Calculation finished, run next cell to visualise error map")
    
gui_error.add_button("run", description="Calculate")
gui_error["run"].on_click(run_error)
gui_error.show()
    

In [ ]:
print("RSE: ", error_map.getRSE())
print("RSP: ", error_map.getRSP())
plt.imshow(error_map.imRSE)

## Let's compare the resolution of the raw data with the SRRF using FRC and DecorrelationAnalysis. Let's start with calculation the FRC resolution of the raw data.

In [ ]:
gui_frc_1 = EasyGui("FRC")

import numpy as np
from nanopyx.core.analysis.frc import FIRECalculator

def run_frc(b):
    gui_frc_1.save_settings()
    pixel_size = gui_frc_1["pixel_size"].value
    units = gui_frc_1["units"].value
    gui_frc_1["run"].disabled = True
    gui_frc_1["run"].description = "Calculating..."
    global frc_calculator_raw
    frc_calculator_raw = FIRECalculator(pixel_size=pixel_size, units=units)
    frc_calculator_raw.calculate_fire_number(dataset_original[3], dataset_original[11])
    gui_frc_1["run"].disabled = False
    gui_frc_1["run"].description = "Calculate"
    print("Calculation finished. You can now plot the results.")
    
gui_frc_1.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=100, remember_value=True)
gui_frc_1.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_frc_1.add_button("run", description="Calculate")
gui_frc_1["run"].on_click(run_frc)
gui_frc_1.show()

In [ ]:
frc_calculator_raw.plot_frc_curve()

## Now let's measure the resolution of the generated SRRF image using FRC

In [ ]:
gui_frc = EasyGui("FRC")

from nanopyx.core.analysis.frc import FIRECalculator

def run_frc(b):
    gui_frc.save_settings()
    pixel_size = gui_frc["pixel_size"].value
    units = gui_frc["units"].value
    first_frame = gui_frc["first_frame"].value
    second_frame = gui_frc["second_frame"].value
    gui_frc["run"].disabled = True
    gui_frc["run"].description = "Calculating..."
    global frc_calculator
    frc_calculator = FIRECalculator(pixel_size=pixel_size, units=units)
    frc_calculator.calculate_fire_number(dataset_srrf[0][first_frame], dataset_srrf[0][second_frame])
    gui_frc["run"].disabled = False
    gui_frc["run"].description = "Calculate"
    print("Calculation finished. You can now plot the results.")
    
gui_frc.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=20, remember_value=True)
gui_frc.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_frc.add_int_slider("first_frame", description="First Frame:", min=0, max=dataset_srrf[0].shape[0]-1, value=0)
gui_frc.add_int_slider ("second_frame", description="Second Frame:", min=0, max=dataset_srrf[0].shape[0]-1, value=1)
gui_frc.add_button("run", description="Calculate")
gui_frc["run"].on_click(run_frc)
gui_frc.show()

In [ ]:
frc_calculator.plot_frc_curve()

## Let's do the same using Decorrelation Analysis.

In [ ]:
gui_decorr_1 = EasyGui("DecorrAnalysis")

from nanopyx.core.analysis.decorr import DecorrAnalysis

def run_decorr(b):
    gui_decorr_1.save_settings()
    pixel_size = gui_decorr_1["pixel_size"].value
    units = gui_decorr_1["units"].value
    rmin = gui_decorr_1["rmin"].value
    rmax = gui_decorr_1["rmax"].value
    gui_decorr_1["run"].disabled = True
    gui_decorr_1["run"].description = "Calculating..."
    global decorr_calculator_raw
    decorr_calculator_raw = DecorrAnalysis(pixel_size=pixel_size, units=units, rmin=rmin, rmax=rmax)
    decorr_calculator_raw.run_analysis(np.mean(dataset_original, axis=0))
    gui_decorr_1["run"].disabled = False
    gui_decorr_1["run"].description = "Calculate"
    print("Calculation finished. You can now plot the results.")
    
gui_decorr_1.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=100, remember_value=True)
gui_decorr_1.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_decorr_1.add_float_slider("rmin", description="Radius Min:", min=0.0, max=0.5, value=0.0)
gui_decorr_1.add_float_slider("rmax", desctiption="Radius Max:", min=0.5, max=1.0, value=1.0)
gui_decorr_1.add_button("run", description="Calculate")
gui_decorr_1["run"].on_click(run_decorr)
gui_decorr_1.show()

In [ ]:
decorr_calculator_raw.plot_results()

## Now let's measure the resolution of the generated SRRF image using DecorrelationAnalysis

In [ ]:
gui_decorr = EasyGui("DecorrAnalysis")

from nanopyx.core.analysis.decorr import DecorrAnalysis

def run_decorr(b):
    gui_decorr.save_settings()
    pixel_size = gui_decorr["pixel_size"].value
    units = gui_decorr["units"].value
    first_frame = gui_decorr["first_frame"].value
    rmin = gui_decorr["rmin"].value
    rmax = gui_decorr["rmax"].value
    gui_decorr["run"].disabled = True
    gui_decorr["run"].description = "Calculating..."
    global decorr_calculator
    decorr_calculator = DecorrAnalysis(pixel_size=pixel_size, units=units, rmin=rmin, rmax=rmax)
    decorr_calculator.run_analysis(dataset_srrf[0][first_frame])
    gui_decorr["run"].disabled = False
    gui_decorr["run"].description = "Calculate"
    print("Calculation finished. You can now plot the results.")
    
gui_decorr.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=100, remember_value=True)
gui_decorr.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_decorr.add_int_slider("first_frame", description="Frame to be used:", min=0, max=dataset_srrf[0].shape[0]-1, value=0)
gui_decorr.add_float_slider("rmin", description="Radius Min:", min=0.0, max=0.5, value=0.0)
gui_decorr.add_float_slider("rmax", desctiption="Radius Max:", min=0.5, max=1.0, value=1.0)
gui_decorr.add_button("run", description="Calculate")
gui_decorr["run"].on_click(run_decorr)
gui_decorr.show()

In [ ]:
decorr_calculator.plot_results()